### Train ETH data to CNN generative network

In [21]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import torch
if torch.cuda.device_count():
    device = 'cuda'
    use_gpu = True
    print('Good to go')
else:
    device = 'cpu'
    use_gpu = False
    print('Using cpu')

Good to go


In [19]:
from ReadData import ReadCurrentAndField_CNN, add_gaussian_noise
import glob
import os 

# TODO zhoujing edit this Data loading 
# print(os.getcwd())
foldername="./Data/"
filepattern = "MagneticField[0-9]*.txt"
train_file_num= 1200
noise = 0.0
#data = ReadFolder(foldername,filepattern)
current,data = ReadCurrentAndField_CNN (foldername,filepattern,train_file_num)

fileList = glob.glob(foldername+filepattern)
position = data[:,0:3,2:18,2:18,2:18]
Bfield = data[:,3:,2:18,2:18,2:18]

# print(fileList)
print(data.shape)
print('current shape', current.shape)
print('Bfield shape', Bfield.shape)
current = add_gaussian_noise(current,noise=noise)
Bfield = add_gaussian_noise(Bfield,noise=noise)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
from Neural_network import Generative_net,Generative_net_test ,ResidualEMNSBlock_3d, BigBlock, weight_init, eMNS_Dataset
###############################################
# Config the neural network
###############################################
num_input = 8
output_shape = (3,16,16,16)
SB_args = (64,64,1,4) # (Cin, Cout, num_repeat, num_block)
BB_args = (2,3) # (scale_factor, num_block)
SB_block = ResidualEMNSBlock_3d 
BB_block = BigBlock
DF = False # whether using divergence free model

Generative_network = Generative_net_test(SB_args, BB_args, SB_block, BB_block, num_input=num_input, output_shape= output_shape)
print(Generative_network)

from torchviz import make_dot
import torch.nn.functional as F
from Training_loop import grad_loss_Jacobain
x = torch.randn(2,8)
y = Bfield[0:2]
preds = Generative_network(x)
print(preds.shape)
loss =   F.l1_loss(preds,y)+grad_loss_Jacobain(preds,y)
        # optimizer.zero_grad() #zero out all of gradient
loss.backward()

make_dot(loss, params=dict(Generative_network.named_parameters()))


### Tune hyperparameters

In [ ]:
from Neural_network import Generative_net, Generative_net_test, ResidualEMNSBlock_3d, BigBlock, weight_init, eMNS_Dataset
from Training_loop_v2 import train_GM
from functools import partial
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.torch import TorchTrainer
from ray.tune.tuner import Tuner
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import ray

# construct dataset
dataset = eMNS_Dataset(
    train_x=current,
    train_y=Bfield
)
# split the dataset to train, validation, test
train_set, valid_set = torch.utils.data.random_split(dataset, [0.9,0.1])

# normailzation
extremes = dataset.train_norm(train_indices = train_set.indices)

tune_schedule = ASHAScheduler(
        metric="loss", # metric to optimize. This metric should be reported with tune.report()
        mode="min",
        max_t=10,
        grace_period=1, # minimum stop epoch
        reduction_factor=2,
    )
param_space = {
    "scaling_config": ScalingConfig(
        num_workers = 1,
        use_gpu = False,
        #resource_per_worker = {"CPU":1, "GPU":1}
    ),
    # You can even grid search various datasets in Tune.
    # "datasets": {
    #     "train": tune.grid_search(
    #         [ds1, ds2]
    #     ),
    # },
    "train_loop_config": {
                'epochs': tune.choice([10]),
                'lr_max': tune.loguniform(1e-4,1e-2),
                'lr_min': tune.loguniform(1e-5,1e-7),
                'batch_size': tune.choice([4,8,16]),
                'L2_norm'   : tune.choice([0]),
                'verbose': False,
                'DF'     : tune.choice([True,False]),
                'schedule': [],
                'grid_space': 16**3,
                'learning_rate_decay': 0.5,
                'skip_spacing': tune.choice([1,2,4]),
                'num_repeat'  : tune.choice([1,2,4]),
                'num_block'   : tune.choice([1,2,3]),
                'maxB'        : extremes[2],
                'minB'        : extremes[3],
                'train_set'   : train_set,
                'valid_set'   : valid_set,
            }

}

train_percents = np.arange(1.0,1.01,0.1)
RMSE_history_end = np.zeros(len(train_percents))
RMSE_val_history_end = np.zeros(len(train_percents))
loss_history_end = np.zeros(len(train_percents))
iter_history_end = np.zeros(len(train_percents))
mse_history_end = np.zeros(len(train_percents))
mse_val_history_end = np.zeros(len(train_percents))
train_stop_epoch = np.zeros(len(train_percents))

################################################
# Train the neural network
################################################

train_loop_config = {
                'epochs': 10,
                'lr_max': 1e-4,
                'lr_min': 2.5e-6,
                'batch_size': 8,
                'L2_norm'   : 0,
                'verbose': False,
                'DF'     : False,
                'schedule': [],
                'grid_space': 16**3,
                'learning_rate_decay': 0.5,
                'skip_spacing': 1,
                'num_repeat'  : 4,
                'num_block'   : 2,
                'maxB'        : extremes[2],
                'minB'        : extremes[3],
                'device'      : device,
                'train_set'   : train_set,
                'valid_set'   : valid_set
                # You can even grid search various datasets in Tune.
                # "datasets": tune.grid_search(
                #         [ds1, ds2]
                #     ),
}

scaling_config = ScalingConfig(
    num_workers = 1,
    use_gpu = use_gpu,
    #resource_per_worker = {"CPU":1, "GPU":1}
)

run_config = RunConfig(checkpoint_config=CheckpointConfig(num_to_keep=1))

# def train_loop_per_worker(params):
#     train_GM(train_set=train_set, valid_set=valid_set,  device=device, config=params)

trainer = TorchTrainer(
    train_loop_per_worker = train_GM,
    train_loop_config = train_loop_config,
    scaling_config = scaling_config,
    run_config = run_config,

)
result = trainer.fit()
# tuner = tune.Tuner(
#     trainer,
#     param_space = param_space,
#     tune_config =tune.TuneConfig(
#         scheduler=tune_schedule,
#         num_samples=10, # number of samples of hyperparameter space
#     ),
#     # run_config = RunConfig(storage_path="./results", name="test_experiment")
# )
    
# tuner.fit()

In [ ]:
from utils import plot_ray_results
plot_ray_results(result, metrics_names=['rmse_train','rmse_val'])

In [ ]:
!tensorboard --logdir=~/ray_results

In [20]:
from Neural_network import Generative_net, Generative_net_test, ResidualEMNSBlock_3d, BigBlock, weight_init, eMNS_Dataset
from Training_loop_v2 import train_GM
from tqdm import tqdm

# construct dataset
dataset = eMNS_Dataset(
    train_x=current,
    train_y=Bfield
)

config = {
    'epochs': 350,
    'lr_max': 5e-4,
    'lr_min': 2.5e-6,
    'batch_size': 8,
    'L2_norm'   : 0,
    'verbose': False,
    'DF'     : False,
    'schedule': [],
    'grid_space': 16**3,
    'learning_rate_decay': 0.5,
    'skip_spacing': 2,
    'num_repeat'  : 1,
    'num_block'   : 3,
    'device'      : device,
}
train_percents = np.arange(1.0,1.01,0.1)
RMSE_history_end = np.zeros(len(train_percents))
RMSE_val_history_end = np.zeros(len(train_percents))
loss_history_end = np.zeros(len(train_percents))
iter_history_end = np.zeros(len(train_percents))
mse_history_end = np.zeros(len(train_percents))
mse_val_history_end = np.zeros(len(train_percents))
train_stop_epoch = np.zeros(len(train_percents))

################################################
# Train the neural network
################################################
index=0
for train_percent in train_percents:
    epoch_stop = 0
    print('train_percent',train_percent)

    # split the dataset to train, validation, test
    train_set, valid_set = torch.utils.data.random_split(dataset, [0.9,0.1])

    # normailzation
    extremes = dataset.train_norm(train_indices = train_set.indices)

    config['maxB'] = extremes[2]
    config['minB'] = extremes[3]
    config['train_set'] = train_set 
    config['valid_set'] = valid_set



    print("----------------------------")
    
    print("----------------------------")
    # test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=True)


    
    RMSE_history, RMSE_val_history, loss_history, iter_history, mse_history, mse_val_history,epoch_stop,Rsquare = train_GM(
        config=config)
        
    
    #save RMSE and loss after early stopping
    RMSE_history_end[index] = RMSE_history[epoch_stop]
    RMSE_val_history_end[index]= RMSE_val_history[epoch_stop]
    loss_history_end[index] = loss_history[epoch_stop]
    iter_history_end[index] = iter_history[epoch_stop]
    mse_history_end[index] = mse_history[epoch_stop]
    mse_val_history_end[index] = mse_val_history[epoch_stop]
    index=index+1
    print('training stop at epoch:',epoch_stop)
    print('training stop at epoch:',Rsquare)


train_percent 1.0
----------------------------
----------------------------
Epoch 0, Iteration 158, loss = 0.1226, l1 loss=0.0660, grad loss=0.0566
Got rmse 5.629888534545898
Got rmse 5.465129852294922

Epoch 1, Iteration 316, loss = 0.1300, l1 loss=0.0756, grad loss=0.0544
Got rmse 5.793348789215088
Got rmse 5.547918796539307

Epoch 2, Iteration 474, loss = 0.1234, l1 loss=0.0698, grad loss=0.0536
Got rmse 5.290348052978516
Got rmse 5.138336181640625

Epoch 3, Iteration 632, loss = 0.1272, l1 loss=0.0736, grad loss=0.0537
Got rmse 5.928671360015869
Got rmse 5.527541637420654

Epoch 4, Iteration 790, loss = 0.1069, l1 loss=0.0541, grad loss=0.0528
Got rmse 5.48106575012207
Got rmse 5.058950424194336

Epoch 5, Iteration 948, loss = 0.1115, l1 loss=0.0585, grad loss=0.0529
Got rmse 5.0115156173706055
Got rmse 4.748825550079346

Epoch 6, Iteration 1106, loss = 0.1121, l1 loss=0.0591, grad loss=0.0531
Got rmse 5.179642677307129
Got rmse 4.768794536590576

Epoch 7, Iteration 1264, loss = 0.

In [ ]:
torch.save(Generative_network, 'EMS_CNN.pt')	# 这里会存储迄今最优模型的参数

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
ave_site = 5
ave_kernel = 1/ave_site*np.ones(ave_site)
loss_history_conv = np.convolve(loss_history.numpy(),ave_kernel,'same')


plt.title('loss')
plt.plot(iter_history,loss_history,'-o')
plt.plot(iter_history,loss_history_conv,'-*')
plt.legend(['loss','loss_conv'])
plt.xlabel('iterations')
plt.ylabel('loss')
# plt.ylim([0,10])
plt.show()

plt.title('Train and Val RMSE(sample_num=1000)')
plt.plot(iter_history[0:epoch_stop],RMSE_history[0:epoch_stop],'-o')
plt.plot(iter_history[0:epoch_stop],RMSE_val_history[0:epoch_stop],'-*')
# plt.plot(2e-5*np.arange(epoch_stop),RMSE_history[0:epoch_stop]*1000,'-o')
# plt.plot(2e-5*np.arange(epoch_stop),RMSE_val_history[0:epoch_stop]*1000,'-*')
# plt.ylim([15,20])
plt.legend(['train CNN','val CNN'])
plt.xlabel('iterations')
plt.ylabel('RMSE(mT)')
# plt.ylim([0,100])
plt.grid()
plt.show()

plt.title('Train and Val loss(sample_num=1000)')
plt.plot(iter_history[0:epoch_stop],mse_history[0:epoch_stop]*1e6,'-o')
plt.plot(iter_history[0:epoch_stop],mse_val_history[0:epoch_stop]*1e6,'-*')
plt.legend(['train CNN','val CNN'])
plt.xlabel('iterations')
plt.ylabel('mse(mT^2)')
plt.grid()
plt.show()
print(epoch_stop)

